In [ ]:
# Import Libraries
import Metrica_IO as mio
import Metrica_Viz as mviz
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Polygon
from math import sqrt
import Metrica_Velocities as mvel

# Set the path and game_id
DATADIR = r'C:\Users\paul9977\Documents\Projects\Soccer Analytics\Friends of Tracking\LaurieOnTracking\sample-data-master\data'
game_id=2

# Let's read in the tracking data and then convert it to metric
tracking_home = mio.tracking_data(DATADIR, game_id, 'Home')
tracking_away = mio.tracking_data(DATADIR, game_id, 'Away')

tracking_home = mio.to_metric_coordinates(tracking_home)
tracking_away = mio.to_metric_coordinates(tracking_away)

# Pull in the event data
events = mio.read_event_data(DATADIR, game_id)
events = mio.to_metric_coordinates(events)

# reverse direction of play so that home team is always going right->left
tracking_home, tracking_away, events = mio.to_single_playing_direction(tracking_home, tracking_away, events)

# Calculate player velocities
tracking_home = mvel.calc_player_velocities(tracking_home,smoothing=True,filter_='moving_average')
tracking_away = mvel.calc_player_velocities(tracking_away,smoothing=True,filter_='moving_average')

In [ ]:
passes = events[events['Type']=='PASS']
forward_passes = []
for index, apass in passes.iterrows():
    if apass['Team'] == 'Away':
        if apass['Start X'] < apass['End X']:
            forward_passes.append(apass)
            
    elif apass['Team'] == 'Home':
        if apass['Start X'] > apass['End X']:
            forward_passes.append(apass)
            
forward_passes = pd.DataFrame(forward_passes)

forward_passes.head()

In [ ]:
cbdistances = []
i=1
for index, row in forward_passes.iterrows():
    begin_pass = row['Start Frame']
    end_pass = row['End Frame']
#     if shot_frame == pass_frame:
#         pass_frame = events.iloc[(index-2)]['Start Frame']
    if row['Team']=='Away':
        # Get locations of center backs
        homedcbx_shot = [tracking_home['Home_2_x'][begin_pass], tracking_home['Home_3_x'][begin_pass]]
        homedcby_shot = [tracking_home['Home_2_y'][begin_pass], tracking_home['Home_3_y'][begin_pass]]
        homedcbx_pass = [tracking_home['Home_2_x'][end_pass], tracking_home['Home_3_x'][end_pass]]
        homedcby_pass = [tracking_home['Home_2_y'][end_pass], tracking_home['Home_3_y'][end_pass]]
        #Calculate Distances
        defender_distance_pass = sqrt((homedcbx_pass[1] - homedcbx_pass[0])**2 + (homedcby_pass[1] - homedcby_pass[0])**2)
        defender_distance_shot = sqrt((homedcbx_shot[1] - homedcbx_shot[0])**2 + (homedcby_shot[1] - homedcby_shot[0])**2)

    else:
        # Get locations of center backs
        homedcbx_shot = [tracking_away['Away_16_x'][begin_pass], tracking_away['Away_17_x'][begin_pass]]
        homedcby_shot = [tracking_away['Away_16_y'][begin_pass], tracking_away['Away_17_y'][begin_pass]]
        homedcbx_pass = [tracking_away['Away_16_x'][end_pass], tracking_away['Away_17_x'][end_pass]]
        homedcby_pass = [tracking_away['Away_16_y'][end_pass], tracking_away['Away_17_y'][end_pass]]
       #Calculate Distances
        defender_distance_pass = sqrt((homedcbx_pass[1] - homedcbx_pass[0])**2 + (homedcby_pass[1] - homedcby_pass[0])**2)
        defender_distance_shot = sqrt((homedcbx_shot[1] - homedcbx_shot[0])**2 + (homedcby_shot[1] - homedcby_shot[0])**2)

    attributes = {
        "Start Frame": begin_pass,
        "End Frame": end_pass,
        "Event Index": index,
        "Team": row['Team'],
        "Defender Distance At Start": defender_distance_pass,
        "Defender Distance At End": defender_distance_shot,
        "Defender Distance Difference": defender_distance_shot-defender_distance_pass,
        "Absolute Difference": abs(defender_distance_shot-defender_distance_pass),
        "Player": row['From']}
    cbdistances.append(attributes)
    print(str(i)+" out of "+str(len(forward_passes))+" Completed.")
    i+=1

df = pd.DataFrame(cbdistances)

In [ ]:
forward_passes_player = df['Player'].value_counts()
total_space_created = df.groupby("Player")['Absolute Difference'].sum().sort_values(ascending=False)

In [ ]:
start = 127118
first_half = 127179
end = 127230

In [ ]:
for i in range(first_half, end):
    fig, ax = mviz.plot_frame(tracking_home.loc[i], tracking_away.loc[i], include_player_velocities=True)
    homedcbx = [tracking_home['Home_2_x'][i], tracking_home['Home_3_x'][i]]
    homedcby = [tracking_home['Home_2_y'][i], tracking_home['Home_3_y'][i]]
    plt.plot(homedcbx, homedcby)
    plt.text((homedcbx[0] + homedcbx[1])/2, (homedcby[0] + homedcby[1])/2, str(round(sqrt((homedcbx[1] - homedcbx[0])**2 + (homedcby[1] - homedcby[0])**2),2)))
#     plt.show()
#     print(round(sqrt((homedcbx[1] - homedcbx[0])**2 + (homedcby[1] - homedcby[0])**2),2))
    plt.savefig(f'Second Half\Creating_Distance_{i}.jpg')

In [ ]:
fig, ax = plt.subplots()
plt.title('Difference in Center Back Distance')
plt.hist(df['Defender Distance Difference'])
plt.axvline(2, color='red', ls='--')
plt.axvline(-2, color='red', ls='--')
plt.axvline(-0.09474573313349148, color='black', ls='--')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.ylabel('Frequency')
plt.xlabel('Distance (m)')

#plt.text(2.75, 4.95,"Average")
#plt.arrow(2.65, 5, -0.25, -0.25, width=0.1, color='black')

plt.savefig("Difference in Center Back Distance Forward Passes with STD.jpg")
plt.show()

In [ ]:
events[events['Subtype'].str.contains("-GOAL", na=False)]

In [ ]:
start = 52941
end = 53075
for i in range(start, end):
    fig, ax = mviz.plot_frame(tracking_home.loc[i], tracking_away.loc[i], include_player_velocities=True)
    homedcbx = [tracking_home['Home_2_x'][i], tracking_home['Home_3_x'][i]]
    homedcby = [tracking_home['Home_2_y'][i], tracking_home['Home_3_y'][i]]
    plt.plot(homedcbx, homedcby)
    plt.text((homedcbx[0] + homedcbx[1])/2, (homedcby[0] + homedcby[1])/2, str(round(sqrt((homedcbx[1] - homedcbx[0])**2 + (homedcby[1] - homedcby[0])**2),2)))
    plt.show()
    print(i)
    print(round(sqrt((homedcbx[1] - homedcbx[0])**2 + (homedcby[1] - homedcby[0])**2),2))
#    plt.savefig(f'Away Goal\Creating_Distance_{i}.jpg')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Create Data
home_shots=(543,290)
away_shots=(421,238)

# create plot
fig, ax = plt.subplots()
index = np.arange(2)
bar_width = 0.35
opacity = 0.8

rects1 = plt.bar(index, home_shots, bar_width,
alpha=0.75,
color='r',
label='Home')

rects2 = plt.bar(index + bar_width, away_shots, bar_width,
alpha=0.75,
color='b',
label='Away')

plt.ylabel('Passes')
plt.title('Breakdown of Passes')
plt.xticks(index+bar_width/2, ("Total", "Foward Passes"))
plt.legend()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()

plt.savefig("Forward Passes.jpeg")